<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-08-15 11:16:18


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS', 'MCDOWELL-N']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='max', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_tmp['InFolio'] = mypf[(mypf['Symbol']) == stock_id.upper()]['InFolio'].values[0]
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell.index = df_weekly_buy_sell.index.date
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria,InFolio
2023-11-27,13.0,9.0,42.78,JPPOWER.NS,23.25,7.60,69.08,80.93,69.0,1.0,Buy,AVSP,1.0
2023-11-27,307.0,216.0,42.08,GICRE.NS,427.20,209.76,46.31,39.20,92.0,1.0,Buy,AVSP,1.0
2023-11-27,267.0,202.0,32.20,HSCL.NS,476.15,190.62,40.10,78.30,69.0,1.0,Buy,AVR,NaN
2023-11-27,665.0,626.0,6.18,LICI.NS,1184.35,601.05,10.59,78.18,62.0,1.0,Buy,AV200,0.0
2023-11-27,652.0,563.0,15.87,MAXHEALTH.NS,948.50,543.25,20.08,45.40,74.0,1.0,Buy,AAR,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05,927.0,961.0,-3.54,IRCTC.NS,1109.05,655.83,41.34,19.65,45.0,-1.0,Sell,AVSP,1.0
2024-08-12,237.0,241.0,-1.68,NIACL.NS,291.73,126.66,87.08,23.12,48.0,-1.0,Sell,AVR,0.0
2024-08-12,57.0,60.0,-4.80,CENTRALBK.NS,68.00,34.95,63.43,19.05,42.0,-1.0,Sell,AVR,NaN
2024-08-12,599.0,665.0,-9.92,AARTIIND.NS,1097.92,452.64,32.33,83.29,42.0,-1.0,Sell,AVR,1.0


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria,InFolio
2024-05-13,122.0,125.0,-2.70,BANKINDIA.NS,148.18,84.23,44.40,21.83,47.0,-1.0,Sell,AVR,1.0
2024-05-27,1574.0,1601.0,-1.70,RADICO.NS,1783.33,1180.29,33.34,13.32,47.0,-1.0,Sell,AV40N,NaN
2024-06-03,119.0,123.0,-3.35,J&KBANK.NS,141.34,86.54,37.36,18.90,44.0,-1.0,Sell,AVR,0.0
2024-06-24,817.0,838.0,-2.47,HAPPSTMNDS.NS,1474.38,741.70,10.19,80.40,47.0,-1.0,Sell,AAR,1.0
2024-06-24,133.0,135.0,-1.36,UNIONBANK.NS,156.07,83.88,58.76,17.20,47.0,-1.0,Sell,AAR,NaN
2024-07-08,182.0,185.0,-1.79,SURYODAY.NS,277.80,152.20,19.37,52.91,47.0,-1.0,Sell,AVR,1.0
2024-07-15,28.0,30.0,-5.05,INFIBEAM.NS,40.09,13.94,104.30,40.77,45.0,-1.0,Sell,AAR,NaN
2024-07-15,229.0,231.0,-0.97,NMDC.NS,280.15,113.79,101.04,22.46,48.0,-1.0,Sell,AVR,0.0
2024-07-29,244.0,249.0,-2.13,BANKBARODA.NS,278.48,184.60,32.02,14.27,46.0,-1.0,Sell,AVR,1.0
2024-08-05,266.0,274.0,-2.96,SHALBY.NS,317.60,222.40,19.56,19.44,47.0,-1.0,Sell,AVSP,1.0


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria,InFolio
2024-06-17,9031.0,7499.0,20.42,CERA.NS,10420.45,6700.09,34.78,15.39,69.0,1.0,Buy,AV40N,NaN
2024-06-17,824.0,745.0,10.55,ROSSARI.NS,1525.49,676.95,21.66,85.22,63.0,1.0,Buy,AVSP,1.0
2024-06-17,4734.0,4459.0,6.16,FINEORG.NS,7060.87,4027.42,17.54,49.16,62.0,1.0,Buy,AV40N,1.0
2024-06-17,163.0,146.0,11.32,CUB.NS,238.73,123.42,31.69,46.88,62.0,1.0,Buy,AAR,0.0
2024-06-24,515.0,468.0,10.01,WIPRO.NS,705.05,378.49,36.03,36.94,62.0,1.0,Buy,AVR,1.0
2024-07-01,1510.0,1384.0,9.13,CLEAN.NS,2570.36,1292.75,16.83,70.19,63.0,1.0,Buy,AVR,1.0
2024-07-01,40.0,37.0,8.76,ITBEES.NS,43.75,32.35,24.39,8.72,65.0,1.0,Buy,AAR,NaN
2024-07-01,781.0,719.0,8.57,ISEC.NS,822.10,586.50,33.09,5.32,62.0,1.0,Buy,AV40N,NaN
2024-07-01,98.0,84.0,16.95,JMFINANCIL.NS,111.27,72.21,36.05,13.26,64.0,1.0,Buy,AVR,1.0
2024-07-01,865.0,813.0,6.37,SPORTKING.NS,1312.31,758.28,14.04,51.75,61.0,1.0,Buy,AAR,0.0


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)